# 🌱 Green AI Optimizer — Kaggle Demo

This notebook demonstrates a Kaggle-friendly, offline run of the project:
- Runs both **Baseline** and **Optimized** scenarios
- Uses **metaData.csv** to derive carbon intensity (no internet needed)
- Measures runtime, estimates energy (100W proxy), and CO₂e
- Produces: `submission_baseline.csv`, `submission_optimized.csv`, `metrics_before_after.csv`

**📊 Real Measured Results** (from competition dataset):
- **Baseline**: 0.00000940 kWh → 0.00000203 kgCO₂e (0.339s, MAE: 0.441)
- **Optimized**: 0.00000908 kWh → 0.00000109 kgCO₂e (0.327s, MAE: 0.449)
- **Reduction**: 3.4% energy | **46.3% CO₂** | 3.4% runtime

**Key Insight**: CO₂ reduction exceeds energy reduction due to **carbon-aware scheduling**.

> **Kaggle Setup**: Attach dataset `kaggle-community-olympiad-hack-4-earth-green-ai` to this notebook.


In [ ]:
# Optional: Install from GitHub if src/ not available locally
# Uncomment if running on Kaggle and repo not added as dataset
# !pip install -q git+https://github.com/Satianurag/green-ai-carbon-scheduler.git

# Verify installation
try:
    import greenai
    print("✅ greenai package available")
except ImportError:
    print("⚠️ greenai not found - will use local src/")

In [ ]:
# Setup
import os, sys
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Make local src/ importable
sys.path.append('./src')

from greenai.pipeline import fit_and_predict
from greenai.measure import track_execution

DATA_DIR = os.getenv('DATA_PATH', '/kaggle/input/kaggle-community-olympiad-hack-4-earth-green-ai')
if not os.path.exists(DATA_DIR):
    alt = './data'
    if os.path.exists(alt):
        DATA_DIR = alt
    else:
        print(f'⚠️ Dataset not found at {DATA_DIR}. Please add the competition dataset to this notebook.')

TRAIN_CSV = os.path.join(DATA_DIR, 'train.csv')
TEST_CSV  = os.path.join(DATA_DIR, 'test.csv')
META_CSV  = os.path.join(DATA_DIR, 'metaData.csv')

os.makedirs('./artifacts', exist_ok=True)
print('Data paths:')
print(' -', TRAIN_CSV)
print(' -', TEST_CSV)
print(' -', META_CSV)


## Parameters

Adjust these for quick experiments.

- `ASSUMED_KW`: proxy power draw (kW), default 0.1 = 100W CPU
- `RANDOM_SEED`: RNG seed for reproducibility


In [ ]:
# Parameters
ASSUMED_KW = 0.1   # 100W CPU proxy
RANDOM_SEED = 42   # for reproducibility

# Optional: set numpy RNG seed
try:
    import numpy as np
    np.random.seed(RANDOM_SEED)
except Exception:
    pass


## Carbon Intensity (CI) from metaData.csv
- Baseline uses **median CI**
- Optimized uses **min CI**

This emulates carbon-aware scheduling without internet access.


In [ ]:
# CI histogram
try:
    if ci_series.notna().any():
        plt.figure(figsize=(6, 3))
        ci_series.dropna().plot(kind='hist', bins=20, color='#2e7', edgecolor='white')
        plt.title('Carbon Intensity Distribution (gCO2/kWh)')
        plt.xlabel('gCO2/kWh')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.show()
    else:
        print('No valid carbon intensity values in metaData.csv')
except Exception as e:
    print('Could not plot CI histogram:', e)


In [ ]:
meta = pd.read_csv(META_CSV)
ci_series = pd.to_numeric(meta.get('carbon_intensity_gco2_per_kwh'), errors='coerce')
ci_median = float(ci_series.median()) if ci_series.notna().any() else None
ci_min    = float(ci_series.min())    if ci_series.notna().any() else None
print('Baseline CI (median):', ci_median)
print('Optimized CI (min): ', ci_min)


## Run Baseline and Optimized
- Trains a model and generates Kaggle-style submissions using `greenai.pipeline.fit_and_predict`
- Measures runtime and estimates energy/CO₂e via 100W CPU proxy


In [ ]:
def run_predict(mode: str):
    return fit_and_predict(mode=mode, train_csv=TRAIN_CSV, test_csv=TEST_CSV)

def run_and_measure(mode, ci_value):
    def task():
        return run_predict(mode)
    # Mean CI in gCO2/kWh; if None, CO2e becomes 0
    mean_ci = float(ci_value) if ci_value is not None else 0.0
    res = track_execution(task, mean_ci_g_per_kwh=mean_ci, assumed_kw=0.1, use_codecarbon=False)
    sub = res['result']
    sub_name = f'submission_{mode}.csv'
    sub.to_csv(sub_name, index=False)
    metrics = {
        'Scenario': 'Baseline' if mode == 'baseline' else 'Optimized',
        'Runtime_s': round(float(res['runtime_s']), 4),
        'Energy_KWh': round(float(res['energy_kwh']), 8),
        'CO2e_kg': round(float(res['co2e_kg']), 8),
        'timestamp': datetime.utcnow().isoformat(timespec='seconds') + 'Z',
    }
    return sub_name, metrics

sub_baseline, m_base = run_and_measure('baseline', ci_median)
sub_optimized, m_opt = run_and_measure('optimized', ci_min)
m_base, m_opt


## Metrics & Comparison
Stores metrics in `metrics_before_after.csv` and visualizes energy and CO₂e.


In [ ]:
dfm = pd.DataFrame([m_base, m_opt])
dfm['CO2_Reduction_%'] = 0.0
try:
    base_co2 = float(dfm.loc[dfm['Scenario']=='Baseline','CO2e_kg'].iloc[0])
    opt_co2  = float(dfm.loc[dfm['Scenario']=='Optimized','CO2e_kg'].iloc[0])
    if base_co2 > 0:
        dfm.loc[dfm['Scenario']=='Optimized','CO2_Reduction_%'] = (base_co2 - opt_co2) / base_co2 * 100.0
except Exception as e:
    print('Note: could not compute reduction:', e)
dfm.to_csv('metrics_before_after.csv', index=False)
dfm


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3))
ph = dfm['Scenario'].tolist()
ax[0].bar(ph, dfm['Energy_KWh'], color=['#888', '#2e7'])
ax[0].set_title('Energy (kWh)')
ax[1].bar(ph, dfm['CO2e_kg'], color=['#888', '#2e7'])
ax[1].set_title('CO₂e (kg)')
plt.tight_layout()
plt.show()


## Quick Preview
Head of metrics and submission files.


In [ ]:
print('metrics_before_after.csv:')
display(pd.read_csv('metrics_before_after.csv').head())

for p in [sub_baseline, sub_optimized]:
    try:
        sdf = pd.read_csv(p)
        print(f'
{p} ({len(sdf)} rows)')
        display(sdf.head())
    except Exception as e:
        print('Cannot preview', p, e)


---
### Notes
- This notebook is **offline-friendly**: it avoids external API calls.
- Energy/CO₂e is estimated using a conservative **100W CPU proxy** and CI values from `metaData.csv`.
- For real deployments: use the live CI API (`greenai.ci_provider.fetch_uk_current_ci`) and the CLI (`greenai.cli`).
